<h2>Importing</h2>

In [51]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import sqlite3
from datetime import datetime

<h2> Initialization</h2>

In [52]:
url="https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
table_name="Countries_by_GDP"
path=r"C:\Users\GPU-TECH\Desktop\Importantجدا\ETL_pipline\Countries_by_GDP.csv"
db_name="World_Economies.db"
log_file='log_file.txt'


<h2>Extract </h2>

<p>This function extracts the required
    information from the website and saves it to a dataframe. The
    function returns the dataframe for further processing. </p>

In [53]:
def extract(url):
    content=requests.get(url).text
    data=BeautifulSoup(content,'html.parser')
    
    tables=data.find_all('tbody')
    rows=tables[2].find_all('tr')
    data_list=[]
    for row in rows:
        col=row.find_all('td')
        if len(col)>=8:
        
            data_dict = {
                'country': col[0].text.strip(),
                'imf_forecast': col[2].text.strip(),
                'worldbank_estimate': col[4].text.strip(),
                'unitednations_estimate': col[6].text.strip(),
            }
            data_list.append(data_dict)
    df=pd.DataFrame(data_list)
    return df
extract(url)

,country,imf_forecast,worldbank_estimate,unitednations_estimate
0,World,"104,476,432","100,562,011","96,698,005"
1,United States,"26,949,643","25,462,700","23,315,081"
2,China,"17,700,899","17,963,171","17,734,131"
3,Germany,"4,429,838","4,072,192","4,259,935"
4,Japan,"4,230,862","4,231,141","4,940,878"
...,...,...,...,...
177,Micronesia,458,427,404
178,Marshall Islands,277,280,257
179,Kiribati,246,223,227
180,Nauru,150,151,155


<h2> Transform </h2>

<p> This function converts the GDP information from Currency
    format to float value, transforms the information of GDP from
    USD (Millions) to USD (Billions) rounding to 2 decimal places.
    The function returns the transformed dataframe.</p>

In [54]:
def transform(df):
    df['imf_forecast'] = df['imf_forecast'].str.replace(',', '').astype(float)
    df['worldbank_estimate'] = df['worldbank_estimate'].str.replace(',', '').astype(float)
    df['unitednations_estimate'] = df['unitednations_estimate'].str.replace(',', '').astype(float)
    df['gdp_usd_billions'] = (df['imf_forecast'] + df['worldbank_estimate'] + df['unitednations_estimate']) / 1000
    df['gdp_usd_billions'] = df['gdp_usd_billions'].round(2)
    return df

<h2>Load</h2>

<p> This function saves the final dataframe as a `CSV` file 
    in the provided path. Function returns nothing.</p>


In [55]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path,index=False)


<h2>Create_db</h2>

<p>This function saves the final dataframe as a database table
    with the provided name. Function returns nothing.</p>


In [56]:
def load_to_db(df,sql_connection ,table_name):
    df.to_sql(table_name,sql_connection,if_exists='replace',index=False)

<h2>Querying the db</h2>

In [57]:
def run_query(query_statement, sql_connection):
    query_output = pd.read_sql(query_statement, sql_connection)
    return query_output

<h2>LOG_Progress</h2>

<p> This function logs the mentioned message at a given stage of the code execution to a log file. Function returns nothing </p>



In [58]:
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) #strftime attribute to convert the timestamp to a string format
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 

<h2>Function calls </h2>

In [61]:
log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df)

log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, path)

log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('World_Economies.db')

log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Running the query')

# Running the Query
query_statement = f"SELECT country,gdp_usd_billions FROM {table_name}"  # Modify the table name to match the one used in the database
result = run_query(query_statement, sql_connection)
print(result)
log_progress('Process Complete.')
sql_connection.close()

              country  gdp_usd_billions
0               World         301736.45
1       United States          75727.42
2               China          53398.20
3             Germany          12761.96
4               Japan          13402.88
..                ...               ...
177        Micronesia              1.29
178  Marshall Islands              0.81
179          Kiribati              0.70
180             Nauru              0.46
181            Tuvalu              0.18

[182 rows x 2 columns]
